In [9]:
import geopandas as gpd
import pandas as pd
from scipy.spatial import KDTree
from sklearn.preprocessing import QuantileTransformer
from sklearn.neighbors import KNeighborsClassifier
from libpysal.graph import read_parquet

In [10]:
clusters_dir = '/data/uscuni-ulce/processed_data/clusters/'
chars_dir = '/data/uscuni-ulce/processed_data/chars/'
regions_datadir = "/data/uscuni-ulce/"
region_hulls = gpd.read_parquet(
        regions_datadir + "regions/" + "cadastre_regions_hull.parquet"
)
graph_dir = '/data/uscuni-ulce/processed_data/neigh_graphs/'

In [78]:
region_id = 69333

In [89]:
def assign_noise(region_id):
    
    # read cluster data and characters
    clusters = pd.read_parquet(f'{clusters_dir}{region_id}_clusters.pq', columns=['final', 'morph'])
    chars = pd.read_parquet(f'{chars_dir}primary_chars_{region_id}.parquet')
    chars = chars[chars.index >= 0]
    assert (clusters.index == chars.index).all()


    # get morphotopes and noise groups values
    is_noise = clusters['final'].values == -1
    morph_medians = chars[~is_noise].groupby(clusters[~is_noise].morph.values).median()
    tq1 = read_parquet(graph_dir + f"tessellation_graph_{region_id}.parquet")
    noise_graph = tq1.subgraph(clusters[is_noise].index)
    noise_groups = chars[is_noise].groupby(noise_graph.component_labels).median()    
    
    ## aggreate morphotope medians and noise in the same dataframe and normalise
    ## cannot use individual tess cells as takes too much time
    all_data = pd.concat((morph_medians, noise_groups))
    scalar = QuantileTransformer(subsample=None, output_distribution='uniform')
    vals = scalar.fit_transform(all_data)
    all_data = pd.DataFrame(vals, index=all_data.index, columns=all_data.columns).fillna(0)


    # morph -> final dictinary for final cluster assignment
    morph_to_final = pd.Series(clusters.loc[~is_noise, 'final'].values,
                               clusters.loc[~is_noise, 'morph'].values)
    morph_to_final = morph_to_final[~morph_to_final.index.duplicated()]
    
    # setup tree data
    query_data = all_data.loc[noise_groups.index]
    tree_data = all_data.loc[morph_medians.index]

    ## query a knn classifier and assign predictions
    tree = KNeighborsClassifier(n_neighbors=15, n_jobs=-1)
    tree = tree.fit(X=tree_data.values, y=morph_to_final.loc[tree_data.index].values)
    predictions = tree.predict(query_data.values)

    #map noise group clusters to building ids
    bid_to_ngroup = noise_graph.component_labels
    bid_to_prediction = bid_to_ngroup.map(pd.Series(predictions, query_data.index).to_dict())
    clusters.loc[bid_to_prediction.index.values, 'final'] = bid_to_prediction.values
    return clusters.final

In [90]:
%%time
for region_id, _ in region_hulls.iterrows():
    print(region_id)
    final_without_noise = assign_noise(region_id)
    clusters = gpd.read_parquet(f'{clusters_dir}{region_id}_clusters.pq')
    # assignment hasnt changed existing clusters
    pd.testing.assert_series_equal(final_without_noise[clusters.final != -1], clusters.final[clusters.final != -1])
    clusters['final_without_noise'] = final_without_noise
    clusters.to_parquet(f'{clusters_dir}{region_id}_clusters.pq')

4
10
132


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (445). n_quantiles is set to n_samples.
  warnings.warn(


134
286


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (234). n_quantiles is set to n_samples.
  warnings.warn(
/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (176). n_quantiles is set to n_samples.
  warnings.warn(


313


...

134454
134755
134905
135687
136169
136533


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (828). n_quantiles is set to n_samples.
  warnings.warn(


137469


/home/krasen/urban_taxonomy/.pixi/envs/default/lib/python3.12/site-packages/sklearn/preprocessing/_data.py:2785: UserWarning: n_quantiles (1000) is greater than the total number of samples (724). n_quantiles is set to n_samples.
  warnings.warn(


137537
137636
137863
138804
139096
139621
139674
139764
140420
141272
141366
142100
143701
145906
146285
147112
147634
148018
148085
149997
150044
151676
152081
152511
CPU times: user 10min 21s, sys: 1min 40s, total: 12min 1s
Wall time: 9min 6s


In [70]:
# %%time
# tree = KDTree(tree_data.values)
# _, idxs = tree.query(query_data.values, k=1, workers=-1)

# clusters.loc[is_noise, 'final'] =  morph_to_final.loc[tree_data.index[idxs]].values

### Plotting

In [91]:
from core.cluster_validation import get_color
final_clusters = pd.read_parquet('/data/uscuni-ulce/processed_data/clusters/clusters_v2.pq')[0]
final_colors = pd.DataFrame(get_color(final_clusters.values), final_clusters.values).drop_duplicates()
final_colors.loc[-1] = [255,255,255]

In [92]:
# region_id = 69333 
region_id = 151676

In [93]:
etcs = gpd.read_parquet(f'{clusters_dir}{region_id}_clusters.pq')

In [94]:
etcs['geometry'] = etcs.simplify(1).to_crs(epsg=4326).make_valid()
etcs = etcs[etcs['geometry'].geom_type == 'Polygon']


In [95]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.SolidPolygonLayer.from_geopandas(etcs, opacity=.7)

CPU times: user 1.24 s, sys: 156 ms, total: 1.4 s
Wall time: 1.4 s


In [96]:
from sidecar import Sidecar
sc = Sidecar(title=f'Final Clusters with noise')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.DarkMatter)
with sc:
    display(m)

In [97]:
from core.cluster_validation import get_color
layer.get_fill_color = final_colors.loc[etcs.final_without_noise].values.astype('uint8')